# 📈 Прогнозирование LTV клиента (Lifetime Value— пожизненная ценность )

**Цель проекта** — построение модели машинного обучения, способной предсказывать пожизненную ценность клиента (LTV) на основе транзакционных и поведенческих данных. Это позволит:

- Сегментировать клиентов по прибыльности.<p>
- Оптимизировать маркетинговые затраты.<p>
- Повысить точность прогнозов выручки.<p>

**🔍Описание проекта**<p>
Набор данных содержит информацию о 100 тысячах реальных заказов, сделанных с 2016 по 2018 год на нескольких торговых площадках Бразилии: от статуса заказа, цены, оплаты и доставки до местоположения клиента, характеристик товара и, наконец, отзывов покупателей.

**данные:** [Brazilian E-Commerce Public Dataset by Olist](https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce) 


<u>olist_customers_dataset.csv</u> -набор данных содержит информацию о клиенте и его местоположении.<p> 
<u>olist_geolocation_dataset.csv</u>  -набор данных содержит информацию о почтовых индексах Бразилии и её координатах (широта и долгота).<p> 
<u>olist_order_items_dataset.csv</u> -набор данных включает данные о товарах, купленных в каждом заказе.<p>
<u>olist_order_payments_dataset.csv</u> -набор данных включает данные о вариантах оплаты заказов.<p>
<u>olist_order_reviews_dataset.csv</u> -набор данных включает данные об отзывах покупателей.<p>
<u>olist_orders_dataset.csv</u> -основной набор данных. В каждом заказе вы можете найти всю остальную информацию.<p>
<u>olist_products_dataset.csv</u> -набор данных включает данные о продуктах, продаваемых Olist.<p>
<u>olist_sellers_dataset.csv</u> -набор данных содержит информацию о продавцах, которые выполнили заказы, сделанные в Olist.<p>
<u>product_category_name_translation.csv</u> -переводит название категории продукта на английский язык.<p>

# Load Data

In [1]:
import pandas as pd
import numpy as np
import os
import shutil
import kagglehub


In [8]:
# Папка назначения

DATA_DIR = os.path.abspath("../data/raw")
os.makedirs(DATA_DIR, exist_ok=True)

# Скачиваем с Kaggle (olistbr/brazilian-ecommerce)

print("Скачиваем датасет с Kaggle...")
kaggle_path = kagglehub.dataset_download("olistbr/brazilian-ecommerce")

# Перемещаем файлы

print("Перемещаем датасеты...")
for filename in os.listdir(kaggle_path):
    src_file = os.path.join(kaggle_path, filename)
    dst_file = os.path.join(DATA_DIR, filename)
#    shutil.copy(src_file, dst_file)

Скачиваем датасет с Kaggle...
Перемещаем датасеты...


In [9]:
# Пути

RAW_PATH = 'data/raw'
PROCESSED_PATH = 'data/processed'

# Создание processed-папки

os.makedirs(PROCESSED_PATH, exist_ok=True)

In [4]:
# Загрузка всех датасетов

customers = pd.read_csv(os.path.join(RAW_PATH, 'olist_customers_dataset.csv'))

orders = pd.read_csv(os.path.join(RAW_PATH, 'olist_orders_dataset.csv'))

order_items = pd.read_csv(
    os.path.join(RAW_PATH, 'olist_order_items_dataset.csv'))

products = pd.read_csv(os.path.join(RAW_PATH, 'olist_products_dataset.csv'))

payments = pd.read_csv(
    os.path.join(RAW_PATH, 'olist_order_payments_dataset.csv'))

reviews = pd.read_csv(os.path.join(RAW_PATH,
                                   'olist_order_reviews_dataset.csv'))
reviews['review_comment_message'].fillna('No comment', inplace=True)
reviews['review_comment_title'].fillna('No title', inplace=True)

sellers = pd.read_csv(os.path.join(RAW_PATH, 'olist_sellers_dataset.csv'))

categories = pd.read_csv(
    os.path.join(RAW_PATH, 'product_category_name_translation.csv'))

/var/folders/h2/dsmg5l1j0ts5vnwjkk8_hcy40000gn/T/ipykernel_12963/680729972.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  reviews['review_comment_message'].fillna('No comment', inplace=True)
/var/folders/h2/dsmg5l1j0ts5vnwjkk8_hcy40000gn/T/ipykernel_12963/680729972.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are

# Preprocessed data

In [5]:
# Приведение столбцов orders с датами к datetime

date_cols_orders = [
    'order_purchase_timestamp', 'order_approved_at',
    'order_delivered_carrier_date', 'order_delivered_customer_date',
    'order_estimated_delivery_date'
]
for col in date_cols_orders:
    orders[col] = pd.to_datetime(orders[col], errors='coerce')

In [10]:
# Объединение с клиентами для получения информации о клиентах.
orders_full = orders.merge(customers, how='left', on='customer_id')

# Объединить с order_items для получения подробной информации о товаре.
orders_full = orders_full.merge(order_items, how='left', on='order_id')

# Объединить с платежами для получения информации о платежах.
orders_full = orders_full.merge(payments, how='left', on='order_id')

# Объединить с товарами для просмотра подробной информации о товаре.
orders_full = orders_full.merge(products, how='left', on='product_id')

# Объединить с категориями для перевода категорий
orders_full = orders_full.merge(categories,
                                how='left',
                                on='product_category_name')
# Объединить с отзывами для анализа настроений
orders_full = orders_full.merge(reviews, how='left', on='order_id')

# Объединить с продавцами для получения информации о продавцах
orders_full = orders_full.merge(sellers, how='left', on='seller_id')

orders_full.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119143 entries, 0 to 119142
Data columns (total 40 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   order_id                       119143 non-null  object        
 1   customer_id                    119143 non-null  object        
 2   order_status                   119143 non-null  object        
 3   order_purchase_timestamp       119143 non-null  datetime64[ns]
 4   order_approved_at              118966 non-null  datetime64[ns]
 5   order_delivered_carrier_date   117057 non-null  datetime64[ns]
 6   order_delivered_customer_date  115722 non-null  datetime64[ns]
 7   order_estimated_delivery_date  119143 non-null  datetime64[ns]
 8   customer_unique_id             119143 non-null  object        
 9   customer_zip_code_prefix       119143 non-null  int64         
 10  customer_city                  119143 non-null  object        
 11  

In [11]:
# Проверка на наличие дубликатов
print(f'Duplicates: {orders_full.duplicated().sum()}')

Duplicates: 0


In [13]:
# Удалить ненужные столбцы
orders_full = orders_full.drop(
    columns=['seller_zip_code_prefix', 'customer_zip_code_prefix'],
    errors='ignore')

In [14]:
orders_full

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_city,...,product_width_cm,product_category_name_english,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,seller_city,seller_state
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,7c396fd4830fd04220f754e42b4e5bff,sao paulo,...,13.0,housewares,a54f0611adc9ed256b57ede6b6eb5114,4.0,No title,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48,maua,SP
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,7c396fd4830fd04220f754e42b4e5bff,sao paulo,...,13.0,housewares,a54f0611adc9ed256b57ede6b6eb5114,4.0,No title,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48,maua,SP
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,7c396fd4830fd04220f754e42b4e5bff,sao paulo,...,13.0,housewares,a54f0611adc9ed256b57ede6b6eb5114,4.0,No title,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48,maua,SP
3,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,af07308b275d755c9edb36a90c618231,barreiras,...,19.0,perfumery,8d5266042046a06655c8db133d120ba5,4.0,Muito boa a loja,Muito bom o produto.,2018-08-08 00:00:00,2018-08-08 18:37:50,belo horizonte,SP
4,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,3a653a41f6f9fc3d2a113cf8398680e8,vianopolis,...,21.0,auto,e73b67b67587f7644d5bd1a52deb1b01,5.0,No title,No comment,2018-08-18 00:00:00,2018-08-22 19:07:58,guariba,SP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119138,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02,da62f9e57a76d978d02ab5362c509660,praia grande,...,40.0,baby,29bb71b2760d0f876dfa178a76bc4734,4.0,No title,So uma peça que veio rachado mas tudo bem rs,2018-03-01 00:00:00,2018-03-02 17:50:01,tupa,SP
119139,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27,737520a9aad80b3fbbdad19b66b37b30,nova vicosa,...,22.0,home_appliances_2,371579771219f6db2d830d50805977bb,5.0,No title,Foi entregue antes do prazo.,2017-09-22 00:00:00,2017-09-22 23:10:57,sao paulo,SP
119140,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15,5097a5312c8b157bb7be58ae360ef43c,japuiba,...,20.0,computers_accessories,8ab6855b9fe9b812cd03a480a25058a1,2.0,No title,Foi entregue somente 1. Quero saber do outro p...,2018-01-26 00:00:00,2018-01-27 09:16:56,ilicinea,MG
119141,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15,5097a5312c8b157bb7be58ae360ef43c,japuiba,...,20.0,computers_accessories,8ab6855b9fe9b812cd03a480a25058a1,2.0,No title,Foi entregue somente 1. Quero saber do outro p...,2018-01-26 00:00:00,2018-01-27 09:16:56,ilicinea,MG


In [15]:
# Сохраняем итоговую таблицу

orders_full.to_csv(os.path.join(PROCESSED_PATH, 'orders_full.csv'),
                   index=False)

**orders_full** содержит всю информацию: пользователь, заказ, товар, оплата, отзывы, продавец. Используем how='left' — сохраняем все строки с заказами, даже если что-то отсутствует (например, отзывов нет). Пропуски не трогаем пока — это задача следующего этапа .